# 속도 정리

- 기간 : 2019.01 ~ 2020.12

- 원본 데이터/csv/2019, 2020 폴더의 .csv 파일 사용

- **폴더명** : csv (xlsx -> csv 변환된 파일들 (xlsx 로드가 너무 오래 걸림))


- **필터링** : 도시고속도로, 차선수 2 이상(차선수 1 제외 하면 될듯)

- 24시 -> 00시로


- 주중, 주말 분리하여 .xlsx 로 월별로 한 시트에 저장

- **파일명** : yyyy_일별_속도(주중 | 평일).xlsx

- ~~2020.12 시트별로 나눠져 있음 ~~



__순서__
1. topis(https://topis.seoul.go.kr/refRoom/openRefRoom_1.do) 에서 '도로별 일자별 통행속도' 데이터 다운
2. csv 파일로 변환
3. 년도 별로 폴더 만들어서 저장
4. 해당 노트의 2번 셀의 yyyy 리스트에 년도 추가 

In [1]:
import pandas as pd
import openpyxl

In [2]:
import os

# 폴더 안의 파일 불러오기 
path = '원본 데이터/csv/'
yyyy = ['2019', '2020', '2021']

In [3]:
file_list = []

for i in range(len(yyyy)):
    file_list.append(os.listdir(path + yyyy[i]))

In [4]:
path_list = []
sheet_name  = []

# file_list 인덱스, yyyy 인덱스를 변경하여 년도 별로 정리
for i in range(len(yyyy)):
    tmp_path_list = []
    tmp_sheet_name  = []
    
    for j in file_list[i]:
        # 경로 + 파일명 
        file_path = path + yyyy[i] + '/' + j
        tmp_path_list.append(file_path)

        # 시트 이름 : 파일명에서 문자열 처리 : yy_mm 형식으로 
        s = j[j.find('년')-2 : j.index('월')]
        s = s.replace('년 ', '_')
        tmp_sheet_name.append(s)
    
    path_list.append(tmp_path_list)
    sheet_name.append(tmp_sheet_name)
    

In [5]:
path_list[len(path_list)-1]

['원본 데이터/csv/2021/2021년 01월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 02월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 03월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 04월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 05월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 06월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 07월 서울시 차량통행속도.csv',
 '원본 데이터/csv/2021/2021년 08월 서울시 차량통행속도.csv']

In [7]:
sheet_name.sort()
sheet_name

[['19_01',
  '19_02',
  '19_03',
  '19_04',
  '19_05',
  '19_06',
  '19_07',
  '19_08',
  '19_09',
  '19_10',
  '19_11',
  '19_12'],
 ['20_01',
  '20_02',
  '20_03',
  '20_04',
  '20_05',
  '20_06',
  '20_07',
  '20_08',
  '20_09',
  '20_10',
  '20_11',
  '20_12'],
 ['21_01', '21_02', '21_03', '21_04', '21_05', '21_06', '21_07', '21_08']]

In [9]:
# 정제된 데이터를 저장할 월간교통량_평균 파일 생성
wb = openpyxl.Workbook()

weekday_writer = []
weekend_writer = []

for year in yyyy:
    # 년도 별로 파일 생성
    weekday_file = year + '_일별_속도(주중)' + '.xlsx'
    weekend_file = year + '_일별_속도(주말)' + '.xlsx'
    
    wb.save(weekday_file)  
    wb.save(weekend_file) 
    
    # excel write 를 위한 writer 정의
    weekday_writer.append(pd.ExcelWriter(weekday_file, engine = 'xlsxwriter'))
    weekend_writer.append(pd.ExcelWriter(weekend_file, engine = 'xlsxwriter'))

In [10]:
for i, j in zip(weekday_writer, weekend_writer):
    print("! ", i)
    print("!! ", j)

!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB7BF9C10>
!!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB5635130>
!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB7F84E80>
!!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB80C0520>
!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB80C08B0>
!!  <pandas.io.excel._xlsxwriter.XlsxWriter object at 0x0000012AB80C0D00>


In [11]:
t = 0
for i in range(len(path_list)):
    print("! " + str(i))
    for j in range(len(path_list[i])):
        print("!! " + str(j) + " " + sheet_name[i][j])
        df = pd.read_csv(path_list[i][j])

        # 필터링 : 도시고속도로, 차선수 2 이상(차선수 1 제외 하면 될듯)
        flt_df = df[(df['차선수'] != 1) & (df['기능유형구분'] == '도시고속도로')]

        # 24시 열 순서 01시 앞으로 변경
        col1 = flt_df.columns[:11].to_list()
        col2 = flt_df.columns[-1:].to_list()
        col3 = flt_df.columns[12:-1].to_list()
        new_col = col1 + col2 + col3
        new_df = flt_df[new_col]

        # 24시 -> 00시로
        result_df = new_df.rename(columns = {'24시':'00시'})

        # 주중, 주말 분리
        weekday_df = result_df[(result_df['요일']!='토') & (result_df['요일']!='일')]
        weekend_df = result_df[(result_df['요일']=='토') | (result_df['요일']=='일')]

        # 개수 확인
        print(len(result_df))
        print(len(weekday_df) + len(weekend_df) == len(result_df))
          
        # 시트별로 파일에 저장
        # 주중 데이터 저장
        weekday_df.to_excel(weekday_writer[i], index = False, sheet_name= sheet_name[i][j])
        # 주말 데이터 저장
        weekend_df.to_excel(weekend_writer[i], index = False, sheet_name= sheet_name[i][j])

    weekday_writer[i].save()
    weekend_writer[i].save() 

! 0
!! 0 19_01
8370
True
!! 1 19_02
7560
True
!! 2 19_03
8370
True
!! 3 19_04
8088
True
!! 4 19_05
8370
True
!! 5 19_06
8100
True
!! 6 19_07
8370
True
!! 7 19_08
8369
True
!! 8 19_09
8096
True
!! 9 19_10
8367
True
!! 10 19_11
8100
True
!! 11 19_12
8370
True
! 1
!! 0 20_01
8240
True
!! 1 20_02
7824
True
!! 2 20_03
8343
True
!! 3 20_04
8074
True
!! 4 20_05
8366
True
!! 5 20_06
8092
True
!! 6 20_07
8360
True
!! 7 20_08
8342
True
!! 8 20_09
8095
True
!! 9 20_10
8346
True
!! 10 20_11
7979
True
!! 11 20_12
8336
True
! 2
!! 0 21_01
8344
True
!! 1 21_02
7555
True
!! 2 21_03
8357
True
!! 3 21_04
7822
True
!! 4 21_05
8184
True
!! 5 21_06
7874
True
!! 6 21_07
8177
True
!! 7 21_08
8172
True


# 한개 먼저 해보기

In [32]:
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147011 entries, 0 to 147010
Data columns (total 36 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   일자       147011 non-null  int64  
 1   요일       147011 non-null  object 
 2   도로명      147011 non-null  object 
 3   링크아이디    147011 non-null  int64  
 4   시점명      147011 non-null  object 
 5   종점명      147011 non-null  object 
 6   방향       147011 non-null  object 
 7   거리       147011 non-null  int64  
 8   차선수      147011 non-null  int64  
 9   기능유형구분   147011 non-null  object 
 10  도심/외곽구분  147011 non-null  object 
 11  권역구분     147011 non-null  object 
 12  01시      146958 non-null  float64
 13  02시      146909 non-null  float64
 14  03시      146753 non-null  float64
 15  04시      146811 non-null  float64
 16  05시      146762 non-null  float64
 17  06시      146698 non-null  float64
 18  07시      146857 non-null  float64
 19  08시      146956 non-null  float64
 20  09시      146977 non-null  

In [45]:
# 도시고속도로는 모두 차선수 2 이상
df[(df['차선수'] == 1) & (df['기능유형구분'] == '도시고속도로')]

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시


In [40]:
# 필터링 : 도시고속도로, 차선수 2 이상(차선수 1 제외 하면 될듯)
flt_df = df[(df['차선수'] != 1) & (df['기능유형구분'] == '도시고속도로')]
flt_df

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
128,20190101,화,강남순환로,1179910010,본선시점부(금천),금천영업소,상행,273,4,도시고속도로,...,42.33,43.00,43.67,44.83,40.50,44.58,43.50,40.58,41.25,39.25
129,20190101,화,강남순환로,1179910020,금천영업소,관악터널입구,상행,118,7,도시고속도로,...,90.00,90.67,87.92,89.42,89.08,90.42,89.50,91.33,95.42,96.50
130,20190101,화,강남순환로,1179910030,관악터널입구,관악터널중간부,상행,2770,3,도시고속도로,...,92.67,92.92,91.50,92.33,93.42,94.25,95.50,98.08,102.08,104.17
131,20190101,화,강남순환로,1209910040,관악터널중간부,관악터널출구부,상행,2088,3,도시고속도로,...,90.00,90.83,90.50,90.08,92.33,92.08,94.17,95.33,99.00,105.00
132,20190101,화,강남순환로,1209910050,관악터널출구부,봉천터널입구부,상행,296,3,도시고속도로,...,81.25,80.67,80.75,81.33,82.17,84.08,85.25,84.92,87.75,92.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145810,20190131,목,올림픽대로,1240005400,강일IC동측(올림픽대로),올림픽대로 서울강동-경기하남,하행,578,4,도시고속도로,...,78.50,77.00,75.50,70.08,58.25,42.67,65.67,71.92,75.17,82.33
146184,20190131,목,자유로,2180000201,가양대교북단IC,강변북로고양시계,상행,227,5,도시고속도로,...,45.83,31.75,34.00,34.25,45.33,55.50,78.08,80.25,81.75,85.58
146185,20190131,목,자유로,1130007400,강변북로고양시계,가양대교북단,상행,238,5,도시고속도로,...,65.17,63.00,63.58,67.83,66.75,69.67,71.00,71.67,73.42,75.42
146186,20190131,목,자유로,1130007300,가양대교북단,강변북로고양시계,하행,240,5,도시고속도로,...,69.92,67.33,64.83,52.75,37.50,38.92,52.00,66.33,66.83,72.42


In [88]:
# 24시 -> 00시로
col1 = flt_df.columns[:11].to_list()
col2 = flt_df.columns[-1:].to_list()
col3 = flt_df.columns[12:-1].to_list()
new_col = col1 + col2 + col3

new_df = flt_df[new_col]
result_df = new_df.rename(columns = {'24시':'00시'})
result_df

,일자,요일,도로명,링크아이디,시점명,종점명,방향,거리,차선수,기능유형구분,도심/외곽구분,00시,01시,02시,03시,04시,05시,06시,07시,08시,09시,10시,11시,12시,13시,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
128,20190101,화,강남순환로,1179910010,본선시점부(금천),금천영업소,상행,273,4,도시고속도로,외곽,39.25,70.00,53.00,43.58,48.20,49.00,36.00,43.60,40.58,45.08,46.58,46.64,43.33,44.67,42.08,42.33,43.00,43.67,44.83,40.50,44.58,43.50,40.58,41.25
129,20190101,화,강남순환로,1179910020,금천영업소,관악터널입구,상행,118,7,도시고속도로,외곽,96.50,73.86,87.42,94.50,94.67,91.75,94.83,92.42,92.67,94.33,92.42,93.00,90.83,91.25,92.75,90.00,90.67,87.92,89.42,89.08,90.42,89.50,91.33,95.42
130,20190101,화,강남순환로,1179910030,관악터널입구,관악터널중간부,상행,2770,3,도시고속도로,외곽,104.17,75.14,93.25,105.42,104.25,102.92,101.08,101.50,101.00,102.17,98.67,97.58,94.67,93.83,96.50,92.67,92.92,91.50,92.33,93.42,94.25,95.50,98.08,102.08
131,20190101,화,강남순환로,1209910040,관악터널중간부,관악터널출구부,상행,2088,3,도시고속도로,외곽,105.00,76.29,91.25,102.83,102.92,102.83,100.67,97.92,99.75,100.50,97.25,96.58,93.17,91.25,91.33,90.00,90.83,90.50,90.08,92.33,92.08,94.17,95.33,99.00
132,20190101,화,강남순환로,1209910050,관악터널출구부,봉천터널입구부,상행,296,3,도시고속도로,외곽,92.58,74.57,82.67,89.67,91.58,91.75,91.33,87.50,88.67,88.17,85.75,84.00,82.42,84.67,82.58,81.25,80.67,80.75,81.33,82.17,84.08,85.25,84.92,87.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145810,20190131,목,올림픽대로,1240005400,강일IC동측(올림픽대로),올림픽대로 서울강동-경기하남,하행,578,4,도시고속도로,외곽,82.33,85.08,86.67,87.58,87.17,85.92,85.83,82.08,80.83,80.58,79.33,77.58,76.00,76.50,76.83,78.50,77.00,75.50,70.08,58.25,42.67,65.67,71.92,75.17
146184,20190131,목,자유로,2180000201,가양대교북단IC,강변북로고양시계,상행,227,5,도시고속도로,외곽,85.58,87.75,88.83,89.42,91.33,89.83,86.75,73.83,47.00,20.75,19.67,20.33,21.08,35.17,46.00,45.83,31.75,34.00,34.25,45.33,55.50,78.08,80.25,81.75
146185,20190131,목,자유로,1130007400,강변북로고양시계,가양대교북단,상행,238,5,도시고속도로,외곽,75.42,76.42,77.33,77.83,78.25,78.58,76.42,69.00,57.67,39.75,33.83,35.75,41.67,56.92,66.67,65.17,63.00,63.58,67.83,66.75,69.67,71.00,71.67,73.42
146186,20190131,목,자유로,1130007300,가양대교북단,강변북로고양시계,하행,240,5,도시고속도로,외곽,72.42,80.17,88.50,89.92,88.58,88.08,86.00,80.33,73.25,70.33,72.17,70.08,67.33,70.50,71.25,69.92,67.33,64.83,52.75,37.50,38.92,52.00,66.33,66.83


In [90]:
# 주중, 주말 분리
weekday_df = result_df[(result_df['요일']!='토') & (result_df['요일']!='일')]
# weekday_df

In [91]:
weekend_df = result_df[(result_df['요일']=='토') | (result_df['요일']=='일')]
# weekend_df

In [89]:
# 개수 확인
len(weekday_df) + len(weekend_df) == len(result_df)

True

In [ ]:
# 시트별로 파일에 저장
# 주중 데이터 저장
# 주말 데이터 저장